In [569]:
import os
import importlib
from datetime import datetime, timedelta
import calendar

import pandas as pd
import numpy as np

import db_oper

In [570]:
str_date = '20180720'
dt_date = datetime.strptime(str_date, '%Y%m%d')
print(dt_date.strftime('%Y-%m-%d'))

dt_date2 = dt_date + timedelta(days=365)

str_date2 = dt_date2.strftime('%Y%m%d')
print(dt_date2.strftime('%Y-%m-%d'))
print(str_date2)

2018-07-20
2019-07-20
20190720


In [571]:
# 시가총액 하위 20% 구하기
sql = """
with
price_date as (
    select min(sdate) as sdate
    from prices
    where sdate >= %s
),
cnt as (
    select count(*) as cnt from prices where sdate = (select sdate from price_date)
),
low20 as (
    select * from prices
    where sdate = (select sdate from price_date)
    order by calc_market_cap
    limit cast((select cnt from cnt) * 0.2 as int)
),
rep_date_latest as (
    select max(rdate) as rdate from reports where rdate < (select sdate from price_date)
),
rep_date as (
    select max(rdate) as rdate from reports where rdate < (select rdate from rep_date_latest)
),
reps as (
    select *
    from reports where rdate = (select rdate from rep_date)
),
candidates as (
    select p.code, c.company, p.sdate, p.close, p.calc_market_cap, p.per, p.psr, p.pcr, p.pbr, r.roa, r.gpa, r.debt_to_equity_ratio
    from low20 p
    join reps r on p.code = r.code
    join companies c on p.code = c.code
    where per is not null and per > 0
)
select * from candidates
""" % str_date

print(type(sql))
low20 = db_oper.select_by_query(sql)
# low20.head()

<class 'str'>


In [572]:
# 각 지표 Ranking 구하기
low20['PER_Rank'] = low20['per'].rank(method='max')
low20['PSR_Rank'] = low20['psr'].rank(method='max')
low20['PCR_Rank'] = low20['pcr'].rank(method='max')
low20['PBR_Rank'] = low20['pbr'].rank(method='max')
low20['ROA_Rank'] = low20['roa'].rank(method='max', ascending=False)
low20['GPA_Rank'] = low20['gpa'].rank(method='max', ascending=False)
low20['s_value'] = low20['PER_Rank'] + low20['PSR_Rank'] + low20['PCR_Rank'] + low20['PBR_Rank'] # 슈퍼가치전략
low20['PBRGPA'] = low20['PBR_Rank'] + low20['GPA_Rank'] # 신마법공식
low20['s_v_m'] = low20['PER_Rank'] + low20['PSR_Rank'] + low20['PBR_Rank'] + low20['GPA_Rank'] # 슈퍼밸류모멘텀

In [573]:
len(low20)

126

In [574]:
# 슈퍼가치전략
super_values = low20.sort_values(['s_value']).head(50)

# 신마법공식 2.0
magic = low20.sort_values(['PBRGPA']).head(30)
# 슈퍼밸류모멘텀
svm = low20.sort_values(['s_v_m']).head(50)

# # 그레이엄 마지막 선물 업그레이드 (야는 하위 20%가 아니얌)
# graham2 =low20[low20['debt_to_equity_ratio'] < 50]
# graham2 = graham2[graham2['roa'] < 5]
# graham2 = graham2[graham2['pbr'] >= 0.2]
# graham2 = graham2.sort_values(['PBR_Rank']).head(30)

In [575]:
sql = """
with
price_date as (
    select min(sdate) as sdate
    from prices
    where sdate >= '%s'
    group by sdate
    having count(*) > 1000
)

select code, sdate, close
from prices
where sdate = (select sdate from price_date)
    and code in ('%s')
""" % (str_date2, "','".join(super_values.code))
sval_next = db_oper.select_by_query(sql)
# sval_next.head()

In [576]:
sval_rslt = super_values.merge(sval_next, how='left', on='code', suffixes=['', '_1'])

In [577]:
# sval_rslt.head()

In [578]:
sval_rslt = sval_rslt[['code', 'company', 'close', 'close_1']]
sval_rslt['stock_cnt'] = 10000000 / len(sval_rslt) / sval_rslt['close']
sval_rslt['buy'] = sval_rslt['stock_cnt'] * sval_rslt['close']
sval_rslt['sell'] = sval_rslt['stock_cnt'] * sval_rslt['close_1']
sval_rslt['gain'] = sval_rslt['sell'] - sval_rslt['buy']

In [579]:
# sval_rslt.head()

In [580]:
print('buy : ', '{:,}'.format(sval_rslt['buy'].sum()))
print('sell : ', '{:,}'.format(sval_rslt['sell'].sum()))
print('gain : ', '{:,}'.format(sval_rslt['gain'].sum()))

buy :  10,000,000.0
sell :  10,763,361.876409758
gain :  1,163,361.8764097593
